# IMPLEMENTING DECODER FROM SCRATCH

## 1. Downloading and Reading the Dataset

First, we download the raw Tiny Shakespeare text file (from Karpathy’s repository) and read it into memory as a single string. This corpus will serve as our training data for the character‐level GPT.


In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O tiny_shakespeare.txt

with open("tiny_shakespeare.txt", "r") as f:
    text = f.read()


--2025-06-07 17:51:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘tiny_shakespeare.txt’

tiny_shakespeare.tx 100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2025-06-07 17:51:58 (92.1 MB/s) - ‘tiny_shakespeare.txt’ saved [1115394/1115394]



## 2. Constructing the Vocabulary and Encoding Characters

Next, we build a character‐level vocabulary from the entire text. We sort all unique characters and assign each character a unique integer index (`stoi`) and also prepare a reverse mapping (`itos`). Then, we define simple `encode`/`decode` functions to translate between strings and integer sequences. Finally, we convert the full text into a 1D tensor of integer indices. This tensor (`data`) will be the raw input for our language modeling pipeline.


In [2]:
import torch

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)


## 3. Splitting Data and Creating Batches

We split the single‐tensor dataset into training (90%) and validation (10%) sequences. To train a Transformer, we need to feed it mini‐batches of shape `(batch_size, block_size)`, where `block_size` is our context window (how many characters the model “sees” at once). The `get_batch()` function randomly samples `batch_size` starting indices, slices out `block_size` characters for `x`, and the next `block_size` characters for `y`. During training, `x` serves as the input context and `y` as the target (the next character at each time step).


In [3]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

block_size = 8
batch_size = 4
torch.manual_seed(1337)

def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_split) - block_size, (batch_size,))
    x = torch.stack([data_split[i : i + block_size] for i in ix])
    y = torch.stack([data_split[i + 1 : i + block_size + 1] for i in ix])
    return x, y


## 4. Toy Bigram Language Model

Before diving into the Transformer architecture, we first implement a **Bigram** model: a single embedding matrix that directly predicts the next character from the current character. This “lookup‐table” approach is extremely simple and serves as a sanity check:

1. Each token index is converted to a “logit vector” of size `[vocab_size]`.
2. We compute cross‐entropy loss against the true next‐character index.
3. The `generate()` method auto‐regressively samples one character at a time, appending it to the context.

Although this model is too weak to capture long‐range dependencies, it helps verify that our data pipeline and training loop work end‐to‐end.


In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramLanguageModel(vocab_size)


### 5. Training the Bigram Model

We now train the bigram model for 10,000 steps using the AdamW optimizer (learning rate = 1e‐3). At each step:
- We sample a batch `(xb, yb)` from `get_batch('train')`.
- We compute `logits, loss = m(xb, yb)` and backpropagate.
- After training, we print the final loss and demonstrate sampling 500 characters from the trained bigram model.

Once this is working, we confirm that the model can at least produce text that somewhat resembles Shakespeare’s character distribution (though it won’t capture grammar or structure).


In [5]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("Final Bigram Loss:", loss.item())
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Final Bigram Loss: 2.527496099472046

y me th'd AUSTowicay grear cfr my thed:
NETI:
TINaiglurbthergneYM.
INAm; ainde g wals slulobus whe

KIZ!

we?
W$d he k
MIV;


IETyoZY st:


Gheay Yatondgrende'TEQUNathinin: ans nd s

OREre.
Qhour im, sof bn I's temrpe thithattersuravWhefe h antr forgg m walos nt kit mo RLORZ'd IFknyor!
Sindis prewhous oodod.
wouprebe bll:CUNTHUSt

Thleaix;
Malesheais whe; lino.
ALI;
CXdaimboth,
batly, tyous w F nig har-
ASerin livin ad:
GRveitho wond ve ack'
3'd gat held inghe beromUCOFFoce ?Y's.
Pat fe, healcof


## 6. Transformer Submodules: Attention, Feed‐Forward, and Blocks

Here we implement the core building blocks of a GPT‐style Transformer:

1. **`Head`**: One head of causal self‐attention. We project the input embeddings to keys (`K`), queries (`Q`), and values (`V`), compute scaled dot‐products, apply a causal (lower‐triangular) mask so each position can only attend to itself and earlier positions, apply softmax to get attention weights, and then weight‐sum the values.
2. **`MultiHeadAttention`**: Runs multiple `Head` instances in parallel, concatenates their outputs, and projects back to the model dimension.
3. **`FeedForward`**: A simple two‐layer MLP with ReLU, which expands from `n_embd` to `4*n_embd` and back.
4. **`Block`**: One Transformer block that applies multi‐head self‐attention + feed‐forward, each followed by a residual connection and layer normalization.


In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)

with open('tiny_shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_split) - block_size, (batch_size,))
    x = torch.stack([data_split[i : i + block_size] for i in ix])
    y = torch.stack([data_split[i + 1 : i + block_size + 1] for i in ix])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """One head of self‐attention."""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * (C**-0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """Multiple heads of self‐attention, concatenated."""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """Simple 2-layer MLP with ReLU nonlinearity."""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Transformer block: multi‐head self‐attention + feed‐forward, with residuals & LayerNorm."""
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


## 8. Full GPT‐Style Language Model and Training Loop

Here we assemble the complete GPT‐style Transformer:

1. **Embeddings**  
   - `token_embedding_table`: Converts input token indices into vectors of dimension `n_embd`.  
   - `position_embedding_table`: Injects positional information for up to `block_size` positions.

2. **Transformer Blocks**  
   - We stack `n_layer` copies of our `Block` (each containing multi‐head attention + feed‐forward).  
   - A final `LayerNorm` ensures stable representations.

3. **Output Head**  
   - `lm_head` projects the final hidden states from dimension `n_embd` back to `vocab_size` logits for each timestep.

4. **Forward Pass**  
   - If `targets` are provided, compute cross‐entropy loss across all positions.  
   - Otherwise, return raw logits for inference.

5. **Generation Method**  
   - Auto‐regressively sample one token at a time, always feeding back the last `block_size` tokens as context.

6. **Training Loop**  
   - We train for `max_iters` steps with AdamW (learning rate = 1e‐3).  
   - Every `eval_interval` steps, we evaluate train/validation loss using `estimate_loss()`.  
   - After training, we generate 2,000 characters of Shakespeare to see how well the model learned character‐level dependencies.


In [7]:
class BigramLanguageModel(nn.Module):
    """Full GPT‐style model (unidirectional Transformer)."""
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)               # (B, T, n_embd)
        pos = torch.arange(T, device=device)                    # (T,)
        pos_emb = self.position_embedding_table(pos)             # (T, n_embd)
        x = tok_emb + pos_emb                                    # broadcast to (B, T, n_embd)
        x = self.blocks(x)                                       # (B, T, n_embd)
        x = self.ln_f(x)                                         # (B, T, n_embd)
        logits = self.lm_head(x)                                 # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
print(f"Model size: {sum(p.numel() for p in m.parameters())/1e6:.2f}M parameters")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Model size: 0.21M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5059
step 300: train loss 2.4196, val loss 2.4338
step 400: train loss 2.3504, val loss 2.3566
step 500: train loss 2.2965, val loss 2.3129
step 600: train loss 2.2410, val loss 2.2500
step 700: train loss 2.2057, val loss 2.2191
step 800: train loss 2.1633, val loss 2.1864
step 900: train loss 2.1244, val loss 2.1510
step 1000: train loss 2.1038, val loss 2.1308
step 1100: train loss 2.0707, val loss 2.1197
step 1200: train loss 2.0377, val loss 2.0800
step 1300: train loss 2.0268, val loss 2.0650
step 1400: train loss 1.9918, val loss 2.0356
step 1500: train loss 1.9697, val loss 2.0293
step 1600: train loss 1.9645, val loss 2.0499
step 1700: train loss 1.9404, val loss 2.0129
step 1800: train loss 1.9095, val loss 1.9951
step 1900: train loss 1.9067, val loss 1.9855
step 2000: train loss 1.8854, val loss 1.9948
step 2100: train 

## 9. Generating Text from the Trained Model

Finally, we start with an empty context (`[0]`) and ask the model to generate 2,000 characters. This sample demonstrates how well our character‐level GPT has learned Shakespeare’s style: you should see long runs of coherent (though imperfect) text that mimics the original corpus.


In [8]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_indices = m.generate(context, max_new_tokens=2000)[0]
print(decode(generated_indices.tolist()))



ROTCUMER:
Tyburforth, bloody,
WhIs migute: you duke I use list. WIthon of where's grande will! savist tought!
Why room upwor alond, liegle. I hone, Iell thou sudd have then strue thus mind,
His by blow, Virdom tow, glingien, yithre spees ssince them Those not.

LUCIO:
Look,----
But thou sging them this my freceimmsed,
By thou sovor conursion that thou sade but grove
the tage encond:
It will Rament me; an your touther,
And havis like to-does, and little spright.

GLOUCESTER:
Rewards thou for Panfessira's bigguards such ways!
What curfort his
will havolss you, as I have the cervirs arled,
Dear my love and pitace unto duly son.

Secome:
Offolk, even thy whose my late all that you by jotly us belies!
Lord, we a-montencry! I

SLARNE:
Day, mave from out prrive And orculing
What confess, temimelyour and stropt;
Secumfospet the gatieus I'll that confence-sting,
But; man't, Rolget
would garnion'd live in which, you, prothre?

CORIOLANUS:
What bonum stravoing, not out be seemmed with
That the b

## 🔧 Model Output Quality and Scope for Improvement

The current GPT decoder implementation produces text, but the quality and coherence are limited. This is expected at this early stage due to the following reasons:

### 🚫 Limitations:
- **Tiny dataset**: The training corpus used (`tiny_shakespeare`) is extremely small (~1MB) and not representative of diverse language patterns.
- **Small model capacity**: The model uses a minimal number of layers and heads (e.g., `n_layer=2`, `n_head=2`) to keep it lightweight and fast for demonstration.
- **Limited training**: Training is done for a small number of iterations with modest batch sizes and learning rates.

### ✅ Future Improvements:
- **Scale up the dataset**: Use a much larger corpus (e.g., OpenWebText, books, or your own domain-specific data).
- **Increase model size**: Add more transformer blocks, heads, and embedding dimensions to increase expressiveness.
- **Train longer with better scheduling**: Train for more epochs with learning rate warm-up, decay, and possibly gradient clipping.
- **Add more training tricks**: Use techniques like label smoothing, dropout tuning, and better tokenization (e.g., SentencePiece or Byte-Pair Encoding instead of character-level).
- **Add encoder side or pre-training/fine-tuning loop**: For a full GPT-style pipeline that can generalize to various NLP tasks.

This project is a great first step in understanding transformer internals and building NLP models from scratch. It lays the groundwork for more advanced, production-grade implementations in the future.
